In [41]:
# 코알라유니브 스터디: 공공공공공경경 - 고주형
# 네이버 신지도 데이터 수집하기
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, quote
import random
import warnings


# driver = webdriver.Chrome("./chromedriver")


options = Options()
options.add_experimental_option("detach", True)
options.add_argument("--disable-blink-features=AutomationControlled")
# options.add_argument("--headless")  # headless 모드로 설정
driver = webdriver.Chrome(options=options)


driver.get("https://map.naver.com/v5/search")

# 팝업 창 제거
# driver.find_element(By.CSS_SELECTOR, "button#intro_popup_close").click()

# 검색창에 검색어 입력하기
search_box = driver.find_element(By.CSS_SELECTOR, "div.input_box>input.input_search")
search_box.send_keys("마라탕")

time.sleep(3)

# 검색버튼 누르기
search_box.send_keys(Keys.ENTER)


time.sleep(10)

# 크롤링
for p in range(20):
    # 5초 delay
    time.sleep(2)

    # 페이지가 로드될 때까지 기다림
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))
    
    # js_script = "document.querySelector(\"body > app > layout > div > div.container > div.router-output > "\
    #             "shrinkable-layout > search-layout > search-list > search-list-contents > perfect-scrollbar\").innerHTML"
    # raw = driver.execute_script("return " + js_script)

    # 요소의 내부 HTML을 가져옴
    raw = element.get_attribute("innerHTML")

    html = BeautifulSoup(raw, "html.parser")

    contents = html.select("div > div.ps-content > div > div > div .item_search")
    for s in contents:
        search_box_html = s.select_one(".search_box")

        name = search_box_html.select_one(".title_box .search_title .search_title_text").text
        print("식당명: " + name)
        try:
            phone = search_box_html.select_one(".search_text_box .phone").text
        except:
            phone = "NULL"
        print("전화번호: " + phone)
        address = search_box_html.select_one(".ng-star-inserted .address").text
        print("주소: " + address)

        print("--"*30)
    # 다음 페이지로 이동
    try:
        next_btn = driver.find_element_by_css_selector("button.btn_next")
        next_btn.click()
    except:
        print("데이터 수집 완료")
        break

# 크롭 웹페이지를 닫음
driver.close()

KeyboardInterrupt: 

In [ ]:
def extract_review():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 리뷰 추출
    rev = []  # 추출한 리뷰 저장
    for i in range(1, 11): # 더보기 누르지 않은 상태로 최대 10개
        try:  # 사진 없는 후기는 div 3번째에 텍스트 위치
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
            time.sleep(2)
            comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰
            rev.append(comment)
        except: # 사진 있는 후기는 div 4번째에 텍스트가 위치
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li['+str(i)+']/div[4]/a').send_keys(Keys.ENTER)
            time.sleep(2)
            comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li['+str(i)+']/div[4]/a/span').text  # 리뷰
            rev.append(comment)
    return rev

In [58]:
# 코알라유니브 스터디: 공공공공공경경 - 고주형
# 네이버 신지도 데이터 수집하기
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, quote
import random
import warnings


# driver = webdriver.Chrome("./chromedriver")

from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 인증 정보가 없거나 유효하지 않은 경우
    if not creds or not creds.valid:
        try:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                raise Exception("No valid credentials available.")
        except Exception as e:
            # 오류가 발생한 경우, 새로운 인증 정보를 얻습니다.
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
            # 새로운 인증 정보를 'token.json'에 저장합니다.
            with open('token.json', 'wb') as token:
                pickle.dump(creds, token)

    return creds

def get_sheet_data(service, spreadsheet_id, range_name):
    """특정 스프레드시트의 지정된 범위에 있는 데이터를 가져옵니다."""
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    values = result.get('values', [])

    # if not values:
    #     print('No data found.')
    # else:
    #     for row in values:
    #         # Row 데이터를 출력합니다. 필요에 따라 여기서 다른 작업을 수행할 수 있습니다.
    #         print(row)

    return values


signup_spreadsheets_id = "1lbU4oHhsne04-VLZrhHO-LK_Ien-ixbor5jgDazd7Zs"
GET_RANGE_NAME = 'group!A261:AK360'  # 예를 들어 Sheet1의 K2부터 K 끝까지

creds = oauthByWeb()
service = build('sheets', 'v4', credentials=creds)
data = []

spreadsheet_ids = get_sheet_data(service, signup_spreadsheets_id, GET_RANGE_NAME)

print(len(spreadsheet_ids))



# 크롬 드라이버 실행

for i in spreadsheet_ids:

    try: 
        options = Options()
        options.add_experimental_option("detach", True)
        options.add_argument("--disable-blink-features=AutomationControlled")
        # options.add_argument("--headless")  # headless 모드로 설정
        # options.add_argument("--headless=new")
        driver = webdriver.Chrome(options=options)


        driver.get("https://map.naver.com/v5/search")

        # 팝업 창 제거
        # driver.find_element(By.CSS_SELECTOR, "button#intro_popup_close").click()

        # 검색창에 검색어 입력하기
        search_box = driver.find_element(By.CSS_SELECTOR, "div.input_box>input.input_search")
        search_box.send_keys(i[5])
        time.sleep(3)
        # 검색버튼 누르기
        search_box.send_keys(Keys.ENTER)

    except Exception as e:
        print(e)
        continue



    try:

        one_value = i

        spreadsheet_id = '1lbU4oHhsne04-VLZrhHO-LK_Ien-ixbor5jgDazd7Zs'

        spreadsheet_lenth = get_sheet_data(service, spreadsheet_id, '전화번호있는_농장!A1:AM')

        start_column = len(spreadsheet_lenth) + 1

        range_name = f'전화번호있는_농장!A{start_column}:AM'

        time.sleep(5)
        # iframe 요소를 식별합니다.
        iframe = driver.find_element(By.CSS_SELECTOR, "iframe#entryIframe")

        # 해당 iframe으로 전환합니다.
        driver.switch_to.frame(iframe)
        # search_result = driver.find_element(By.CSS_SELECTOR, "body")
        # html_search_result = search_result.page_source
        # soup = BeautifulSoup(html, 'html.parser')
        # phone_number = driver.find_element(By.CSS_SELECTOR, ".O8qbU.nbXkr .xlx7Q")
        # # phone_number = soup.select(".xlx7Q")

        # number_data = phone_number.text.strip()

        # 현재 페이지의 HTML을 가져옵니다.
        html = driver.page_source

        # BeautifulSoup을 사용하여 HTML을 파싱합니다.
        soup = BeautifulSoup(html, 'html.parser')

        # 원하는 요소를 선택하고 데이터를 가져옵니다.
        phone_number = soup.select_one(".O8qbU.nbXkr .xlx7Q")
        number_data = phone_number.text.strip()

        print(phone_number.text.strip())

        one_value.insert(5, number_data)


        body = {'values':[one_value]}

        # values().update 메소드를 사용하여 스프레드시트에 데이터를 채워넣습니다.
        # valueInputOption='USER_ENTERED'는 사용자가 직접 입력하는 것과 같은 형식으로 데이터를 처리합니다.
        request = service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=range_name,
            valueInputOption='USER_ENTERED',
            body=body
        )
        response = request.execute()

    except Exception as e:
        print(e, "e 검색결과가 없음")
        # driver.close()
        continue
    finally:
        driver.close()



100
Message: no such element: Unable to locate element: {"method":"css selector","selector":"iframe#entryIframe"}
  (Session info: chrome=123.0.6312.124); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001008684a4 chromedriver + 4326564
1   chromedriver                        0x000000010086096c chromedriver + 4295020
2   chromedriver                        0x000000010048c088 chromedriver + 278664
3   chromedriver                        0x00000001004cea80 chromedriver + 551552
4   chromedriver                        0x00000001005074f8 chromedriver + 783608
5   chromedriver                        0x00000001004c34e4 chromedriver + 505060
6   chromedriver                        0x00000001004c3f5c chromedriver + 507740
7   chromedriver                        0x000000010082ba40 chromedriver + 4078144
8   chromedriver                     

In [ ]:
# if __name__ == "__main__":
#     for array in range_array:
#         update_data_list = []
#         for page in range(1, 6):
#             crawling_list = crawlingData(page, array[1])
#             update_data_list += crawling_list

#         creds = oauth()
#         service_sheets = build('sheets', 'v4', credentials=creds)
#         update_sheets_by_2_dimension_list(service_sheets, SPREADSHEET_ID, array[0], update_data_list)

[' ']


In [ ]:
def update_sheets_by_2_dimension_list(service, spreadsheet_id, range_name, values):
    # 스프레드시트 서비스 객체 생성 (인증 과정은 생략)
    
    body = {'values':values}

    # values().update 메소드를 사용하여 스프레드시트에 데이터를 채워넣습니다.
    # valueInputOption='USER_ENTERED'는 사용자가 직접 입력하는 것과 같은 형식으로 데이터를 처리합니다.
    request = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=range_name,
        valueInputOption='USER_ENTERED',
        body=body
    )
    response = request.execute()
